# AnTeDe Lab C: Viterbi Algorithm for HMM

## Session goal

The goal of this session is reproduce the example in Chapter 8 of **Speech and Language Processing** by Daniel Jurafsky & James H. Martin.

We hardcode the same data used in the example.


In [2]:
import nltk.tokenize as tokenize

sentence = "Janet will back the bill"

observations = ("Janet", "will", "back", "the", "bill")
observations = tokenize.word_tokenize(sentence)

states = ("NNP", "MD", "VB", "JJ", "NN", "RB", "DT")
starting_p_values = [0.2767, 0.0006, 0.0031, 0.0453, 0.0449, 0.0510, 0.2026]

starting_p = {}

for i, key in enumerate(states):
    starting_p[key] = starting_p_values[i]

print("Starting Probabilities:")
print(starting_p)

valid_transitions = {}

valid_transitions["NNP"] = [0.3777, 0.0110, 0.0009, 0.0084, 0.0584, 0.0090, 0.0025]
valid_transitions["MD"] = [0.0008, 0.0002, 0.7968, 0.0005, 0.0008, 0.1698, 0.0041]
valid_transitions["VB"] = [0.0322, 0.0005, 0.0050, 0.0837, 0.0615, 0.0514, 0.2231]
valid_transitions["JJ"] = [0.0366, 0.0004, 0.0001, 0.0733, 0.4509, 0.0036, 0.0036]
valid_transitions["NN"] = [0.0096, 0.0176, 0.0014, 0.0086, 0.1216, 0.0177, 0.0068]
valid_transitions["RB"] = [0.0068, 0.0102, 0.1011, 0.1012, 0.0120, 0.0728, 0.0479]
valid_transitions["DT"] = [0.1147, 0.0021, 0.0002, 0.2157, 0.4744, 0.0102, 0.0017]

transition_p = {}

for key in valid_transitions:
    transition_p[key] = {}
    for i, state in enumerate(states):
        transition_p[key][state] = valid_transitions[key][i]

print("Transition Probabilities:")
print(transition_p)

valid_emissions = {}
valid_emissions["NNP"] = {
    "Janet": 0.000032,
    "the": 0.000048,
}
valid_emissions["MD"] = {
    "will": 0.308431,
}
valid_emissions["VB"] = {
    "will": 0.000028,
    "back": 0.000672,
    "bill": 0.000028,
}
valid_emissions["JJ"] = {
    "back": 0.000340,
}
valid_emissions["NN"] = {
    "will": 0.000200,
    "back": 0.000223,
    "bill": 0.002337,
}
valid_emissions["RB"] = {
    "back": 0.010446,
}
valid_emissions["DT"] = {
    "the": 0.506099,
}
emission_p = {}

for key in valid_emissions:
    emission_p[key] = {}
    for word in observations:
        try:
            emission_p[key][word] = valid_emissions[key][word]
        except:
            emission_p[key][word] = 0

print("Emmission Probabilities:")
print(emission_p)

Starting Probabilities:
{'NNP': 0.2767, 'MD': 0.0006, 'VB': 0.0031, 'JJ': 0.0453, 'NN': 0.0449, 'RB': 0.051, 'DT': 0.2026}
Transition Probabilities:
{'NNP': {'NNP': 0.3777, 'MD': 0.011, 'VB': 0.0009, 'JJ': 0.0084, 'NN': 0.0584, 'RB': 0.009, 'DT': 0.0025}, 'MD': {'NNP': 0.0008, 'MD': 0.0002, 'VB': 0.7968, 'JJ': 0.0005, 'NN': 0.0008, 'RB': 0.1698, 'DT': 0.0041}, 'VB': {'NNP': 0.0322, 'MD': 0.0005, 'VB': 0.005, 'JJ': 0.0837, 'NN': 0.0615, 'RB': 0.0514, 'DT': 0.2231}, 'JJ': {'NNP': 0.0366, 'MD': 0.0004, 'VB': 0.0001, 'JJ': 0.0733, 'NN': 0.4509, 'RB': 0.0036, 'DT': 0.0036}, 'NN': {'NNP': 0.0096, 'MD': 0.0176, 'VB': 0.0014, 'JJ': 0.0086, 'NN': 0.1216, 'RB': 0.0177, 'DT': 0.0068}, 'RB': {'NNP': 0.0068, 'MD': 0.0102, 'VB': 0.1011, 'JJ': 0.1012, 'NN': 0.012, 'RB': 0.0728, 'DT': 0.0479}, 'DT': {'NNP': 0.1147, 'MD': 0.0021, 'VB': 0.0002, 'JJ': 0.2157, 'NN': 0.4744, 'RB': 0.0102, 'DT': 0.0017}}
Emmission Probabilities:
{'NNP': {'Janet': 3.2e-05, 'will': 0, 'back': 0, 'the': 4.8e-05, 'bill': 0}, 'M

The following cell contains an implementation of a Viterbi decoder for an HMM.


In [4]:
def viterbi(observations, states, starting_p, transition_p, emission_p):
    # your trellis is a list of dictionaries
    trellis = [{}]

    # first column of the trellis:
    # how likely you are to start in each state, multiplied by
    # how likely you are to generate the initial observation
    # from each state
    for state in states:
        trellis[0][state] = {
            "probability": starting_p[state] * emission_p[state][observations[0]],
            "previous state": None,
        }

    # for loop over the trellis columns, left to right
    for k in range(1, len(observations)):
        # add a column
        new_column = {}

        # for each row in the column
        for state in states:
            max_path_p = 0

            for previous_state in states:
                up_to_here_p = (
                    trellis[k - 1][previous_state]["probability"]
                    * transition_p[previous_state][state]
                )

                if up_to_here_p > max_path_p:
                    max_path_p = up_to_here_p

                    prev_st_selected = previous_state

            max_p = max_path_p * emission_p[state][observations[k]]

            new_column[state] = {"probability": max_p, "previous": prev_st_selected}

        trellis.append(new_column)

    return trellis

Your turn: run **viterbi** and obtain the trellis for our HMM.


In [5]:
my_trellis = [{}]
# BEGIN_YOUR_CODE


# END_YOUR_CODE
for i, column in enumerate(my_trellis):
    print(observations[i])
    print(column)

Janet
{'NNP': {'probability': 8.8544e-06, 'previous state': None}, 'MD': {'probability': 0.0, 'previous state': None}, 'VB': {'probability': 0.0, 'previous state': None}, 'JJ': {'probability': 0.0, 'previous state': None}, 'NN': {'probability': 0.0, 'previous state': None}, 'RB': {'probability': 0.0, 'previous state': None}, 'DT': {'probability': 0.0, 'previous state': None}}
will
{'NNP': {'probability': 0.0, 'previous': 'NNP'}, 'MD': {'probability': 3.00406859104e-08, 'previous': 'NNP'}, 'VB': {'probability': 2.2313087999999997e-13, 'previous': 'NNP'}, 'JJ': {'probability': 0.0, 'previous': 'NNP'}, 'NN': {'probability': 1.03419392e-10, 'previous': 'NNP'}, 'RB': {'probability': 0.0, 'previous': 'NNP'}, 'DT': {'probability': 0.0, 'previous': 'NNP'}}
back
{'NNP': {'probability': 0.0, 'previous': 'MD'}, 'MD': {'probability': 0.0, 'previous': 'MD'}, 'VB': {'probability': 1.6085273254449314e-11, 'previous': 'MD'}, 'JJ': {'probability': 5.106916604768e-15, 'previous': 'MD'}, 'NN': {'probabil

Your turn: write a function **get_pos_tag_sequence** that accepts the Viterbi-decoded trellis as input and prints out the tag for each word in the test sentence. Write a function that backtracks from the last column of the trellis to the first one to follow the most likely path


In [6]:
def get_pos_tag_sequence(trellis):
    # BEGIN_YOUR_CODE




    # END_YOUR_CODE

In [7]:
get_pos_tag_sequence(my_trellis)

Janet	NNP
will	MD
back	VB
the	DT
bill	NN
